In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from fuzzywuzzy import fuzz
import evaluate
import csv
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import top_k_accuracy_score, ndcg_score
import random

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/yelp/splits/train.jsonl"
df_recommender_train = pd.read_json(input_file, lines=True)
for _, row in df_recommender_train.iterrows():
    row["recommended_place"]["place_name"] = row["recommended_place"]["place_name"].lower()

In [4]:
df_recommender_train

,user_id,user_previous_interactions,recommended_place,negative_recommended_place,turns
0,_BHTC7nyCBoZcfiiD5cOXg,"[{'place_name': 'Amish Country Store', 'busine...","{'place_name': 'laziz', 'business_id': 'wS5eU8...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,y5qEpAfJQaOjjG8ZJWrp6w,"[{'place_name': 'Trader Joe's', 'business_id':...","{'place_name': 'vincente's restaurant', 'busin...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,6s-g2vFu12OemhiK3FJuOQ,"[{'place_name': 'Rouge', 'business_id': '4tU9h...","{'place_name': 'cafe saigon', 'business_id': '...","[{'place_name': 'Pho Ha Saigon', 'business_id'...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,Zqu-tMR9CHjA44sU7I5bXA,"[{'place_name': 'Caribbean Delight', 'business...","{'place_name': 'morimoto', 'business_id': '6_T...",[{'place_name': 'Zento Contemporary Japanese C...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,sZUC9GbwvDHWrHl0D_d2dA,"[{'place_name': 'Saturn Club', 'business_id': ...","{'place_name': 'silk city diner & lounge', 'bu...","[{'place_name': 'Parc', 'business_id': 'j-qtdD...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
9738,WwulXySQN8t2hwqH_yWurA,"[{'place_name': 'McDonald's', 'business_id': '...","{'place_name': '3 in 1 restaurant', 'business_...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
9739,61yERKmEa_0mkZYy_3PNAg,"[{'place_name': 'Mesa Verde', 'business_id': '...","{'place_name': 'cajun kitchen cafe', 'business...","[{'place_name': 'State & Fig', 'business_id': ...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
9740,3CwMwbY9UgP-qvTDxNs_0g,"[{'place_name': 'Reed's Dairy - Meridian', 'bu...","{'place_name': 'modern bar and restaurant', 'b...","[{'place_name': 'Sid's Garage', 'business_id':...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
9741,wXdbkFZsfDR7utJvbWElyA,"[{'place_name': 'Chin Brothers Restaurant', 'b...","{'place_name': 'ale emporium', 'business_id': ...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [5]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/yelp/splits/val.jsonl"
df_recommender_validation = pd.read_json(input_file, lines=True)
for _, row in df_recommender_validation.iterrows():
    row["recommended_place"]["place_name"] = row["recommended_place"]["place_name"].lower()

In [6]:
df_recommender_validation

,user_id,user_previous_interactions,recommended_place,negative_recommended_place,turns
0,6kJFLAHV-tNsBEZaRTqEWQ,"[{'place_name': 'Nuts To You', 'business_id': ...","{'place_name': 'devon seafood grill', 'busines...","[{'place_name': 'The Dandelion', 'business_id'...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,sraTrYU-7q_bQ0TxQrLJvQ,"[{'place_name': 'IKEA Restaurant', 'business_i...","{'place_name': 'gen x tavern', 'business_id': ...","[{'place_name': 'Luv Child', 'business_id': 'C...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,-G7Zkl1wIWBBmD0KRy_sCw,"[{'place_name': 'Dew Inn', 'business_id': 'VSW...","{'place_name': 'lau kee restaurant', 'business...","[{'place_name': 'Jane G's Restaurant', 'busine...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,wzUFZ3DaU7k9R2S54IMPfg,"[{'place_name': 'Keegan's Seafood Grille', 'bu...",{'place_name': 'willie jewell’s old school bar...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,gVO3VZwCu54Otc7bUB33Mg,"[{'place_name': 'Chapel Tavern', 'business_id'...","{'place_name': 'ohana sushi', 'business_id': '...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
2082,SGPCLi-kVZ_pz1w7VJtkeA,"[{'place_name': 'Cafe La Maude', 'business_id'...","{'place_name': 'emmy squared - queen village',...","[{'place_name': 'Village Whiskey', 'business_i...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2083,3CwMwbY9UgP-qvTDxNs_0g,"[{'place_name': 'Shady Acres Cafe', 'business_...","{'place_name': 'boise pie co', 'business_id': ...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2084,6jjHo9Lilv3kTy87pm2ycw,"[{'place_name': 'Heritage American Bistro', 'b...","{'place_name': 'tampa taco company', 'business...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2085,iJQtNnEcrxdpaD132v26nQ,"[{'place_name': 'Relaxing Station', 'business_...","{'place_name': 'merci montecito', 'business_id...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
prompt_train = []
recommend_train = []
not_founds = 0

for _, row in tqdm(df_recommender_train.iterrows(), total=len(df_recommender_train)):
    prompt = ""
    found = False
    recommended = row["recommended_place"]["place_name"]
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
        
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_train.append(prompt)
                recommend_train.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"len(prompt_train): {len(prompt_train)}")
print(f"len(recommend_train): {len(recommend_train)}")

            
            
prompt_encodings = tokenizer(prompt_train, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_train, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_train = Dataset.from_dict(dataset)

100%|██████████| 9743/9743 [00:26<00:00, 368.99it/s]


Could not find 49
len(prompt_train): 9694
len(recommend_train): 9694


In [9]:
prompt_validation = []
recommend_validation = []
not_founds = 0

for _, row in tqdm(df_recommender_validation.iterrows(), total=len(df_recommender_validation)):
    prompt = ""
    found = False
    recommended = row["recommended_place"]["place_name"]
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
            
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_validation.append(prompt)
                recommend_validation.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1
        
print(f"Could not find {not_founds}")
print(f"len(prompt_validation): {len(prompt_validation)}")
print(f"len(recommend_validation): {len(recommend_validation)}")
            
            
prompt_encodings = tokenizer(prompt_validation, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_validation, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_validation = Dataset.from_dict(dataset)

100%|██████████| 2087/2087 [00:05<00:00, 380.06it/s]


Could not find 19
len(prompt_validation): 2068
len(recommend_validation): 2068


In [10]:
def data_collator(batch):
    input_ids, attention_mask, labels,  = [], [], []
    for sample in batch:
        input_ids.append(sample['input_ids'])
        attention_mask.append(sample['attention_mask'])
        labels.append(sample['labels'])
    max_encoder_len = max(sum(x) for x in attention_mask)
    max_decoder_len = max(sum([0 if item == IGNORE_INDEX else 1 for item in x]) for x in labels)
    return {
        'input_ids': torch.tensor(input_ids)[:, :max_encoder_len],
        'attention_mask': torch.tensor(attention_mask)[:, :max_encoder_len],
        'labels': torch.tensor(labels)[:, :max_decoder_len]
    }

In [11]:
training_args = TrainingArguments(
    output_dir="/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/yelp/T5_recommender",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=0.3,#self.cfg.save_steps,
    eval_steps=0.3, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_validation,
        data_collator=data_collator,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
trainer.train() # resume_from_checkpoint=True
trainer.save_model()

Step,Training Loss,Validation Loss
1212,3.303500,3.047287
2424,2.927000,2.908435
3636,2.687000,2.871719


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


# Load the model and test it on the test dataset

In [13]:
torch.cuda.empty_cache()

In [14]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/yelp/splits/test.jsonl"
df_recommender_test = pd.read_json(input_file, lines=True)
for _, row in df_recommender_test.iterrows():
    row["recommended_place"]["place_name"] = row["recommended_place"]["place_name"].lower()

In [15]:
apps_training_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/yelp/yelp_df.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(row["name"].lower())
        
all_apps = list(set(all_apps))

In [16]:
all_apps

['top deck restaurant',
 'aura ultra lounge',
 'royal carriages',
 'hilton garden inn nashville/franklin cool springs',
 "ten's showclub",
 'twin dragon',
 'café central',
 "world's fair donuts",
 'made with love juicery',
 'point restaurant & bar',
 'glen mills veterinary hospital',
 'noodles & company',
 'banh mi boys',
 'nam phuong',
 'darfons restaurant & lounge',
 'j&j south philly pizza',
 'the jelly donut',
 'pigeon head brewery',
 "ashton's bed and breakfast",
 'rj chocolatier',
 'wasabi japanese steakhouse and sushi',
 "clay's creative corner bakery",
 "wong's wok",
 "winckler's ribs",
 'rge rd',
 'missouri history museum',
 'soap',
 'boathouse grill',
 "bertucci's italian restaurant",
 "mo'joe coffeehouse",
 'symbowl - kirkwood',
 'sol luna lounge',
 'tandoor & tikka',
 'calabria brickoven pizzeria',
 'urban outfitters',
 'bella trattoria',
 "battista's calzone",
 'hyatt place nashville downtown',
 'new orleans hambuger & seafood',
 "zemam's",
 "cafe lombardi's",
 "not your a

In [17]:
def candidate_creator(row):
    np.random.seed(row.name)
    selected_values = np.random.choice(np.setdiff1d(all_apps, [row["recommended_place"]["place_name"]]), 24, replace=False)
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, row["recommended_place"]["place_name"]) 

df_recommender_test['candidate'] = df_recommender_test.apply(lambda row: candidate_creator(row), axis=1)

In [18]:
prompt_test = []
recommend_test = []
candidate_books = []
true_candidate_indexes = []
not_founds = 0
for _, row in df_recommender_test.iterrows():
    candidates = []
    for index, candidate_book in enumerate(row["candidate"].tolist()):
        candidates.append(candidate_book)
        if candidate_book == row["recommended_place"]["place_name"]:
            true_candidate_index = index
    prompt = ""
    
    found = False
    recommended = row["recommended_place"]["place_name"] 
    
    for index, turn in enumerate(row["turns"]):
        computer = turn["COMPUTER"]
        
        if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
            prompt += "computer: I would recommend the "
            prompt_test.append(prompt)
            recommend_test.append(recommended)
            candidate_books.append(candidates)
            true_candidate_indexes.append(true_candidate_index)
            found = True
            break
        else:
            prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"Number of prompt: {len(prompt_test)}")
print(f"Number of generations: {len(recommend_test)}")
print(f"Number of candidate apps: {len(candidate_books)}")
print(f"Number of true candidate indexes: {len(true_candidate_indexes)}")

Could not find 11
Number of prompt: 2078
Number of generations: 2078
Number of candidate apps: 2078
Number of true candidate indexes: 2078


In [19]:
prompt_test[10]

"computer: Hello! How may I assist you today?\nhuman: I'm looking for an Indian place in Upper Darby. Any suggestions?\ncomputer: Sure! Are there any specific features you’re looking for?\nhuman: I'm interested in places that prioritize delicious food.\ncomputer: Do you prefer to pay using a credit card?\nhuman: Yes, paying by credit card is a must.\ncomputer: Do you need a parking spot for your car or bike?\nhuman: Parking does not affect me.\ncomputer: Sure, what about the number of reviews?\nhuman: The number of reviews doesn't matter much to me.\ncomputer: I would recommend the "

In [20]:
recommend_test[10]

'maroosh halal'

In [21]:
candidate_books[10]

['goodwill industries of middle tennessee',
 'nørdik',
 'clearwater bagels',
 'goleta beach park',
 "daffy's",
 "arnaud's restaurant",
 'frontier',
 'stix & ice',
 'rooster & the till',
 'brewhouse historical sports bar',
 "salvo's seafood & deli",
 'anytime fitness',
 'da-kine hawaiian cafe',
 'deuces bbq',
 'vip market',
 "rocky & carlo's restaurant & bar",
 'the mockingbird nashville',
 'maroosh halal',
 "o'michael's pub & grill",
 'publix super markets',
 'saint benjamin brewing company',
 'hilton new orleans/st. charles avenue',
 'tie dye grill',
 'kiwi blue salon',
 'soho sushi burrito']

In [22]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/yelp/T5_recommender")
model.eval()
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70):
  prompt_batches = list(chunk(prompt_test, batch_size))
  generation_batches = list(chunk(recommend_test, batch_size))

  correctly_predicted = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to('cuda'), attention_mask=inputs["attention_mask"].to('cuda'),
                            max_new_tokens=32,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    decoded_generations = [tokenizer.decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True) for generation in generations_predicted]
    generation_batch = [generation for generation in generation_batch]
    
    correctly_predicted.extend([1 if fuzz.ratio(predicted, ground_truth) > threshold else 0 for predicted, ground_truth in zip(decoded_generations, generation_batch)])

  return correctly_predicted

In [24]:
correctly_predicted = evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

  0%|          | 0/260 [00:00<?, ?it/s]

100%|██████████| 260/260 [04:22<00:00,  1.01s/it]

success_rate:  0.01828681424446583


In [25]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]
    
def convert_to_sublists(numbers, sublist_size):
    return [numbers[i:i+sublist_size] for i in range(0, len(numbers), sublist_size)]

def recommender_rank(prompts, candidate_apps, model, tokenizer, batch_size=8):
  model.eval()
  encoder_max_length = 1024
  decoder_max_length = 32
  prompts_tokenized = tokenizer(prompts, max_length=encoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
  
  input_ids_decoder = []
  attention_mask_decoder = []
  input_ids_encoder = []
  attention_mask_encoder  = []
  for index, candidate_app_elements in enumerate(candidate_apps):
    candidate_app_elements = [tokenizer.pad_token+element for element in candidate_app_elements] # adding pad token to the beginning of each candidate app
    candidate_apps_tokenized = tokenizer(candidate_app_elements, max_length=decoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
    for candidate_app_index in range(len(candidate_app_elements)):
      input_ids_decoder.append(candidate_apps_tokenized["input_ids"][candidate_app_index])
      attention_mask_decoder.append(candidate_apps_tokenized["attention_mask"][candidate_app_index])
      input_ids_encoder.append(prompts_tokenized["input_ids"][index])
      attention_mask_encoder.append(prompts_tokenized["attention_mask"][index])
  
  input_ids_encoder_batches = list(chunk(input_ids_encoder, batch_size))
  attention_mask_encoder_batches = list(chunk(attention_mask_encoder, batch_size))
  input_ids_decoder_batches = list(chunk(input_ids_decoder, batch_size))
  attention_mask_decoder_batches = list(chunk(attention_mask_decoder, batch_size))
  

  scores = []
  for input_ids_encoder_batch, attention_mask_encoder_batch, input_ids_decoder_batch, attention_mask_decoder_batch in tqdm(zip(input_ids_encoder_batches, attention_mask_encoder_batches, input_ids_decoder_batches, attention_mask_decoder_batches), total = len(input_ids_encoder_batches)):
    decoder_input_ids = torch.stack(input_ids_decoder_batch).to("cuda")
    decoder_attention_mask = torch.stack(attention_mask_decoder_batch).to("cuda")
    input_ids = torch.stack(input_ids_encoder_batch).to("cuda")
    attention_mask = torch.stack(attention_mask_encoder_batch).to("cuda")
    with torch.no_grad():
      model_output = model(decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, 
                           input_ids=input_ids, attention_mask=attention_mask)
    
    logprobs = F.log_softmax(model_output["logits"], dim=-1)[:, :-1, :] # remove the eos token
    output_tokens = decoder_input_ids[:, 1:] # remove the bos token
        
    tokens_logprobs = torch.gather(logprobs, 2, output_tokens[:, :, None]).squeeze(-1).to(torch.float32)
        
    mask = torch.ones(tokens_logprobs.shape, dtype=torch.bool, device="cuda")
    for i, _output in enumerate(output_tokens):
      for j, _token in enumerate(_output):
        if _token == tokenizer.pad_token_id:
          mask[i, j] = False
              
    score = (tokens_logprobs * mask).sum(-1) / mask.sum(-1)
    scores.extend(score.to('cpu').tolist())
    
  # batch_input_representations = torch.cat(batch_input_representations)
  
  scores = convert_to_sublists(scores, len(candidate_apps[0]))
  
  return scores

In [26]:
scores = recommender_rank(prompt_test, candidate_books, model, tokenizer, batch_size=8)

100%|██████████| 6494/6494 [24:27<00:00,  4.43it/s]


Sampled Candidates

In [27]:
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

[np.float64(0.19634263715110684),
 np.float64(0.2959576515880655),
 np.float64(0.36910490856592876),
 np.float64(0.4177093358999038),
 np.float64(0.46823869104908566),
 np.float64(0.5101058710298364),
 np.float64(0.5538979788257941),
 np.float64(0.5948026948989413),
 np.float64(0.6400384985563041),
 np.float64(0.6713185755534168)]

In [28]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [29]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]

[np.float64(0.19634263715110684),
 np.float64(0.2591927136618343),
 np.float64(0.29576634215076597),
 np.float64(0.3166991296220906),
 np.float64(0.33624655250930285),
 np.float64(0.35115994292239133),
 np.float64(0.36575731218771057),
 np.float64(0.3786613134036812),
 np.float64(0.3922786471825138),
 np.float64(0.4013206172068943)]

Similar Candidates

In [ ]:
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

In [44]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [ ]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]